In [15]:
from src.loan.constant.trainingpipeline import SCHEMA_FILE_PATH
from sklearn.model_selection import train_test_split
import pandas as pd
import os, sys
from src.loan.exception import ModelException
from src.loan.logger import logging
from scipy.stats import ks_2samp
from src.loan.utils.main_utils import  load_object, load_numpy_array_data,save_object,read_yaml_file

In [3]:
from src.loan.utils.main_utils import read_yaml_file, write_yaml_file

In [3]:
schema_config = read_yaml_file("D:/Machine Learning project/Loan/Loan Project/loan_prediction/config/schema.yaml")

In [4]:
import yaml

def read_yaml_file(filename):
    with open(filename, 'r') as file:
        data = yaml.safe_load(file)
    return data

In [5]:
a = read_yaml_file("D:/Machine Learning project/Loan/Loan Project/loan_prediction/config/schema.yaml")

In [6]:
filename = "D:/Machine Learning project/Loan/Loan Project/loan_prediction/research"

In [7]:
report = {"not":"ok","oy":"jok"}

In [8]:
def write_yaml_file(filename: str, data, replace:bool = False) -> None:
    try:
        if replace:
            if os.path.exists(filename):
                os.remove(filename)
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, "w") as file:
            return yaml.dump(data,file,default_flow_style=False)
    except Exception as e:
        raise ModelException(e,sys)

In [10]:
from src.loan.constant.trainingpipeline import PIPELINE_NAME, ARTIFACT_DIR

In [29]:
a:str = 'data_validation'
b:str = "drift_report"
c:str = "report.yaml"

In [37]:
d = os.path.join(a,b,c)

In [31]:
os.makedirs(d,exist_ok=True)

In [38]:
report = {}

In [39]:
for col in range(3):
    report.update({col:{"p_value": float(col+1),
                                        "drift_status": col -1}})

In [40]:
print(report)

{0: {'p_value': 1.0, 'drift_status': -1}, 1: {'p_value': 2.0, 'drift_status': 0}, 2: {'p_value': 3.0, 'drift_status': 1}}


In [41]:
e = os.path.join(d)

In [2]:
from src.loan.constant.trainingpipeline import TRAIN_FILE_PATH, TEST_FILE_NAME
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras import layers

In [48]:
def build_model(X_train):
        try:
            logging.info("ANN Model building Start")
            model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Input layer with 64 neurons and ReLU activation
    tf.keras.layers.Dense(128, activation='relu'),  # Hidden layer with 128 neurons and ReLU activation
    tf.keras.layers.Dense(64, activation='relu'),   # Hidden layer with 64 neurons and ReLU activation
    tf.keras.layers.Dense(32, activation='relu'),   # Hidden layer with 32 neurons and ReLU activation
    tf.keras.layers.Dense(16, activation='relu'),   # Hidden layer with 16 neurons and ReLU activation
    tf.keras.layers.Dense(8, activation='relu'),    # Hidden layer with 8 neurons and ReLU activation
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer with 1 neuron (for binary classification) and Sigmoid activation
])
            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
            logging.info("Model Building is Complete")
            return model
        except Exception as e:
            raise ModelException(e,sys)

In [56]:
class ANNClassifier:
    def __init__(self,X_train,y_train):
        try:
            self.X_train = X_train
            self.y_train = y_train
            self.model = self._build_model
        except Exception as e:
            raise ModelException(e,sys)
        
    def _build_model(self):
        try:
            logging.info("ANN Model building Start")
            model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(self.X_train.shape[1],)),  # Input layer with 64 neurons and ReLU activation
    tf.keras.layers.Dense(128, activation='relu'),  # Hidden layer with 128 neurons and ReLU activation
    tf.keras.layers.Dense(64, activation='relu'),   # Hidden layer with 64 neurons and ReLU activation
    tf.keras.layers.Dense(32, activation='relu'),   # Hidden layer with 32 neurons and ReLU activation
    tf.keras.layers.Dense(16, activation='relu'),   # Hidden layer with 16 neurons and ReLU activation
    tf.keras.layers.Dense(8, activation='relu'),    # Hidden layer with 8 neurons and ReLU activation
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer with 1 neuron (for binary classification) and Sigmoid activation
])
            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
            logging.info("Model Building is Complete")
            return model
        except Exception as e:
            raise ModelException(e,sys)
    
    def train(self,epochs=100,batch_size=32,validation_data=None):
        logging.info("Model Training is start ")
        #early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
        history = self.model.fit(self.X_train,self.y_train,epochs=epochs,
                                 batch_size=batch_size)#,validation_data=validation_data,
                                 #callbacks=[early_stopping])
        logging.info(f"Model trainig is complete {history.history}")
        return history
    
    def evaluate(self, X_test, y_test):
        logging.info("Model evaluation done")
        return self.model.evaluate(X_test, y_test)

In [5]:
train = os.path.join(X_path)

In [6]:
train=load_numpy_array_data(X_path)

In [7]:
gg = pd.DataFrame(train[:,-1])

In [160]:
gg.nunique()

0    2
dtype: int64

In [8]:
X_train, y_train =(
    train[:,:-1],
    train[:,-1],
)

In [31]:
X_train.shape[1]

396

In [32]:
len(X_train[0])

396

In [162]:
model = ANNClassifier(X_train=X_train,y_train=y_train)

In [163]:
acc = model._build_model()

c:\Users\Nishant Borkar\anaconda3\envs\lven\lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [164]:
acc.fit(X_train,y_train)

12429/12429 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.8027 - loss: 0.4192


In [191]:
acc.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_43 (Dense)                │ (None, 64)             │        25,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,213 (524.27 KB)

 Trainable params: 44,737 (174.75 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 89,476 (349.52 KB)

In [195]:
a = X_train[0]

In [202]:
aa = tf.cast(a, tf.float32)

In [189]:
tf.saved_model.save(acc,"D:/Machine Learning project/Loan/Loan Project/loan_prediction/research/model")

In [165]:
apred =acc.predict(X_train)

12429/12429 ━━━━━━━━━━━━━━━━━━━━ 9s 681us/step


In [95]:
from src.loan.ml.model.metrics import get_classification_score

In [170]:
import numpy as np
import pandas as pd
pred = np.where(apred >= 0.5,1,0)

In [1]:
import pandas as pd 
df = pd.read_csv("D:/Machine Learning project/Loan/Loan Project/loan_prediction/artifact/data_ingestion/data_ingestion/test.csv")

In [2]:
df.head(2)

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
0,555485,31,15,single,rented,yes,Magistrate,Sikar,Rajasthan,12,11,0
1,7804668,54,0,single,rented,no,Comedian,Bhilai,Chhattisgarh,0,14,0


In [41]:
aa = np.array(a[:-1])

In [69]:
aa

array([184333, 3239090, 68, 17, 'single', 'owned', 'no', 'Comedian',
       'Amravati', 'Maharashtra', 13, 11], dtype=object)

In [8]:
taa = df.drop(columns='Risk_Flag',axis=1)

In [121]:
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score

In [172]:
print(f1_score(y_true=y_train,y_pred=pred))

0.9262800370350825


In [112]:
softmax_probs = np.array([[5.9827487e-10], [2.0308545e-10], [1.3390944e-06], [4.1535868e-09], [2.2659113e-07], [1.0400553e-06]])

In [114]:
softmax_probs.squeeze()

array([5.9827487e-10, 2.0308545e-10, 1.3390944e-06, 4.1535868e-09,
       2.2659113e-07, 1.0400553e-06])

In [16]:
new_model = tf.keras.models.load_model("D:/Machine Learning project/Loan/Loan Project/loan_prediction/artifact/model_trainer/trained_model/model.h5")

In [17]:
new_model.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

In [18]:
new_model.fit(X_train,y_train,epochs=3)

Epoch 1/3
12431/12431 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.9343 - loss: 0.1998
Epoch 2/3
12431/12431 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.9342 - loss: 0.1997
Epoch 3/3
12431/12431 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.9354 - loss: 0.1967


In [4]:
from src.loan.pipeline.stage_01_data_ingestion_pipeline import IngestionTrainPipeline
import os


In [8]:
pwd

'd:\\Machine Learning project\\Loan\\Loan Project\\loan_prediction'

In [9]:
from src.loan.pipeline.stage_01_data_ingestion_pipeline import IngestionTrainPipeline
import os
if __name__ == '__main__':
    trainig = IngestionTrainPipeline()
    aa = trainig.start_data_ingestion()
    #logging.info("All is ok")

In [11]:
aa.trained_file_path

'artifact\\data_ingestion\\data_ingestion\\train.csv'

In [46]:
import tensorflow as tf
from src.loan.constant.trainingpipeline import *
import os, sys
from src.loan.utils.main_utils import  load_object
from src.loan.exception import ModelException
from src.loan.logger import logging

class Prediction:
    def __init__(self):
        self.main_dir = ARTIFACT_DIR
        self.model_dir = MODEL_TRAINER_DIR_NAME
        self.model_traind_model = MODEL_TRAINER_TRAINED_MODEL_DIR
        self.model_name = MODEL_TRAINER_TRAINED_MODEL_NAME
        self.data_transform_dir = DATA_TRANSFORMATION_DIR_NAME
        self.transform_object_path = DATA_TRANSFORMATION_TRANSFORMED_OBJECT_DIR
        self.transform_object_name= PREPROCSSING_OBJECT_FILE_NAME
        
        self.transfrom = os.path.join(self.main_dir,self.data_transform_dir,self.transform_object_path,self.transform_object_name)
        self.file  = os.path.join(self.main_dir,self.model_dir,self.model_traind_model,self.model_name)
        self.model = tf.keras.models.load_model(self.file)
        self.preprocesor = load_object(self.transfrom)
    def predict(self, data):
        preprocesor = self.preprocesor
        model = self.model
        pre_data = preprocesor.transform(data)
        pred = model.predict(pre_data)
        return(pred)

In [50]:
aaa = aa.reshape(-1, 1)

In [56]:
import pandas as pd
jk = pd.DataFrame(aa)

In [57]:
jk

,0
0,184333
1,3239090
2,68
3,17
4,single
5,owned
6,no
7,Comedian
8,Amravati
9,Maharashtra


In [63]:
X = np.array([184333, 3239090, 68, 17,'single', 'owned', 'no', 'Comedian','Amravati', 'Maharashtra', 13, 11])

In [71]:
X.reshape(1,-1)

array([['184333', '3239090', '68', '17', 'single', 'owned', 'no',
        'Comedian', 'Amravati', 'Maharashtra', '13', '11']], dtype='<U11')

In [73]:
prediction = Prediction()
pred = prediction.predict(data =X.reshape(1,-1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step


c:\Users\Nishant Borkar\anaconda3\envs\lven\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


In [75]:
pred.argmax()

0

In [14]:
from src.loan.constant.trainingpipeline import MODEL_TRAINER_DIR_NAME, MODEL_TRAINER_TRAINED_MODEL_DIR,MODEL_TRAINER_TRAINED_MODEL_NAME,ARTIFACT_DIR

In [16]:
file = os.path.join(ARTIFACT_DIR,MODEL_TRAINER_DIR_NAME,MODEL_TRAINER_TRAINED_MODEL_DIR,MODEL_TRAINER_TRAINED_MODEL_NAME)